# 🎭 Checkpoint 3: Sistema Completo de Agentes

## Objetivo
Este é o notebook final, onde juntamos todas as peças: os agentes com suas personalidades (Checkpoint 1) e as ferramentas que lhes dão poderes (Checkpoint 2). O objetivo é criar um sistema autônomo que conduz a avaliação do início ao fim.


## 📦 Configuração de ambiente

> *Vamos carregar tudo que fizemos nos Checkpoints 1 e 2!*


In [ ]:
!pip install agno

In [ ]:
import os
from typing import Dict
from google.colab import userdata


os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
print("✅ Ambiente configurado!")


## 📚 Definindo o material

> Este texto é o "cérebro" ou a "fonte da verdade" para os nossos agentes. O Coordenador usará este material para criar perguntas, e o Avaliador o usará como gabarito para julgar as respostas do aluno. Isso garante que a IA se atenha ao escopo do que queremos ensinar.

In [ ]:
material = """
# Git - Controle de Versão

Git é um sistema de controle de versão que registra mudanças no código ao longo do tempo.

## Conceitos Principais

**Repositório (repo):** Pasta que contém seu projeto e todo histórico de mudanças.

**Commit:** Registro de uma mudança no código. Como um "save point" do seu trabalho.
Cada commit tem mensagem explicando o que mudou.

**Branch (ramificação):** Linha independente de desenvolvimento.
- main/master: branch principal (código estável)
- feature branches: para desenvolver novas funcionalidades

**Comandos básicos:**
- git init: cria novo repositório
- git add: marca arquivos para próximo commit
- git commit -m "mensagem": salva mudanças com descrição
- git status: mostra estado atual (o que mudou)
- git log: histórico de commits
- git branch: lista/cria branches
- git checkout: muda de branch
- git merge: junta mudanças de uma branch em outra

**Workflow típico:**
1. Modificar arquivos
2. git add (preparar mudanças)
3. git commit (salvar mudanças)
4. git push (enviar para servidor remoto)

**Por que usar Git?**
- Histórico completo de mudanças
- Trabalho em equipe sem conflitos
- Experimente código sem medo (sempre pode voltar)
- Backup automático do trabalho
"""

## 🤖 Criação dos agentes

- name: Dá um nome ao agente, útil para identificação.
- model=Gemini(id="gemini-2.5-flash"): Especifica o modelo a ser usado. O "flash" é uma versão rápida e eficiente.
- instructions=[...]: Esta é a parte mais crítica. É o prompt do sistema, ou a "personalidade" do agente.

>Diferença Chave: Note como as instructions (o prompt do sistema) dentro das funções criar_agente_... estão muito mais detalhadas do que no Checkpoint 1.

Agora, elas descrevem um fluxo de trabalho passo a passo, mencionando explicitamente os nomes das ferramentas que o agente deve usar (fazer_pergunta, solicitar_avaliacao, etc.) e em que ordem. Estamos ensinando ao agente não apenas quem ele é, mas como ele deve trabalhar.

In [ ]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools import tool
from datetime import datetime


# Funções dos agentes (versão simplificada para demo)
def criar_agente_coordenador(material: str) -> Agent:
    return Agent(
        name="Coordenador Pedagógico",
        model=Gemini(id="gemini-2.5-flash"),
        instructions=[
            f"MATERIAL BASE:\n{material}\n",
            "=" * 60,
            "",
            "COORDENADOR PEDAGÓGICO",
            "",
            f"Avaliar conhecimento do aluno em até 3 perguntas.",
            "",
            "PERGUNTAS: conversacionais.",
            "Use também: 'Por que', 'Como', 'Quando' (não apenas 'O que é?')",
            "Seja contextual: referencie o que o aluno disse antes.",
            "Exemplo: 'Ótimo! Você mencionou X. Agora, por que Y é importante?'",
            "",
            "PROCESSO:",
            "",
            "1. PLANEJAMENTO: Identifique 4-5 tópicos do material",
            "",
            "2. CICLO INTERATIVO (para cada pergunta):",
            "",
            "REGRA: MAX 2 perguntas por tópico. Se BOA->1 e avança. Se PARCIAL/RUIM->máx 2 e avança.",
            "",
            "  a) obter_contexto_recente() - vê histórico recente",
            "  b) Formule pergunta contextual baseada no passo (a)",
            "     - Se é follow-up: referencie a resposta anterior",
            "     - Seja conversacional, não interrogatório",
            "  c) fazer_pergunta(pergunta)",
            "  d) solicitar_avaliacao(pergunta, resposta, topico)",
            "  e) exibir_feedback(feedback_breve) - OBRIGATÓRIO",
            "  f) registrar_decisao(decisao, topico, motivo)",
            "",
            "  Repita até limite ou cobrir todos tópicos.",
            "",
            "3. RELATÓRIO FINAL:",
            "",
            "  a) Obtenha o histórico completo - OBRIGATÓRIO",
            "  b) Gere relatório com EXATAMENTE estes títulos:",
            "     # Relatório de Avaliação",
            "     ## O que você já domina",
            "     ## Onde você pode crescer",
            "     ## Precisa revisar",
            "     ## Seu Plano de Ação",
            "  c) Salve o relatório.",
            "",
            "REGRAS CRÍTICAS:",
            "",
            "FEEDBACK: Breve (2-3 frases), específico, encorajador.",
            "Tom: validador e caloroso, não frio ou mecânico.",
            "Exemplos: 'Correto! Você captou a essência.' / 'Ótimo começo! Continue assim.'",
            "Exiba o feedback após CADA avaliação.",
            "",
            "RELATÓRIO: Use SEMPRE segunda pessoa ('você', 'seu/sua').",
            "NUNCA: 'o aluno', 'ele/ela'. Fale COM o aluno, não SOBRE.",
            "",
            "LIMITE: Se fazer_pergunta retornar erro='LIMITE_ATINGIDO', PARE e gere relatório.",
            "",
            "PERSONALIDADE: Tutor conversacional e encorajador.",
            "Fale COM o aluno como um mentor amigável, não como um interrogador frio.",
        ]
    )

def criar_agente_avaliador(material: str) -> Agent:
    return Agent(
        name="Avaliador Rigoroso",
        model=Gemini(id="gemini-2.5-flash"),
        instructions=[
            f"MATERIAL DE REFERÊNCIA:\n{material}\n",
            "=" * 50,
            "",
            "VOCÊ É O AVALIADOR que avalia respostas dos alunos.",
            "",
            "SUA MISSÃO:",
            "Comparar a resposta do aluno com o material base.",
            "Classificar em uma de três categorias:",
            "",
            "CRITÉRIOS DE CLASSIFICAÇÃO:",
            "",
            "🚫 **nao_sabe**:",
            "- Aluno diz explicitamente 'não sei', 'não lembro', etc",
            "- Resposta completamente incorreta ou sem sentido",
            "- Não demonstra nenhum conhecimento sobre o tópico",
            "",
            "⚠️ **sabe_parcial**:",
            "- Resposta vaga ou incompleta",
            "- Menciona alguns pontos mas falta precisão",
            "- Conceito básico correto mas sem detalhes",
            "- Resposta muito curta (< 10 palavras)",
            "",
            "✅ **sabe_bem**:",
            "- Resposta alinhada com o material",
            "- Inclui detalhes ou exemplos",
            "- Demonstra compreensão real do conceito",
            "- Explicação própria mas correta",
            "",
            "INSTRUÇÕES:",
            "1. Compare a resposta COM O MATERIAL (não com conhecimento externo)",
            "2. Seja JUSTO mas RIGOROSO",
            "3. Para 'proximo_passo':",
            "   - Use 'aprofundar' se conhecimento parcial e < 2 perguntas no tópico",
            "   - Use 'avançar' se domínio OU >= 2 perguntas no tópico",
            "4. Responda SEMPRE neste formato JSON:",
            "{",
            '  "nivel": "nao_sabe" | "sabe_parcial" | "sabe_bem",',
            '  "justificativa": "Explicação curta do porquê",',
            '  "feedback_sugerido": "Feedback amigável para o aluno",',
            '  "proximo_passo": "avançar" | "aprofundar"',
            "}",
            "",
            "IMPORTANTE: Responda APENAS com o JSON, nada mais.",
        ]
    )

# Criar agentes
coordenador = criar_agente_coordenador(material)
avaliador = criar_agente_avaliador(material)

# Estado compartilhado
_estado = {
    "agente_avaliador": avaliador,
    "historico": [],
    "tracking": {},
    "num_perguntas": 0,
    "max_perguntas": 6  # Limite para demo
}


print("✅ Sistema completo carregado!")
print("🤖 Agentes: Coordenador + Avaliador")
print("🔧 Tools: fazer_pergunta + solicitar_avaliacao + exibir_feedback + obter_contexto_recente + obter_historico_completo + salvar_relatorio + registrar_decisao")
print("📊 Estado: Configurado e pronto")
print("⚙️ Função configurar_estado: Disponível para setup")


## 🧰 Tools (Ferramentas)

@tool: Este é um "decorador". Quando ativado, ele sinaliza para a biblioteca agno que esta função não é uma função comum, mas sim uma ferramenta que o agente de IA pode escolher executar.

In [ ]:
@tool
def fazer_pergunta(pergunta: str) -> Dict:
    """Faz pergunta ao usuário e obtém resposta."""
    if _estado["num_perguntas"] >= _estado["max_perguntas"]:
        return {
            "erro": "LIMITE_ATINGIDO",
            "mensagem": f"Limite de {_estado['max_perguntas']} perguntas atingido"
        }

    _estado["num_perguntas"] += 1
    num = _estado["num_perguntas"]

    print(f"\n🤖 TUTOR (Pergunta {num}): {pergunta}")
    resposta = input("👤 VOCÊ: ")

    _estado["historico"].append({
        "tipo": "pergunta",
        "numero": num,
        "conteudo": pergunta
    })

    _estado["historico"].append({
        "tipo": "resposta",
        "numero": num,
        "conteudo": resposta
    })

    return {
        "resposta": resposta,
        "numero_pergunta": num,
        "perguntas_restantes": _estado["max_perguntas"] - num
    }

@tool
def solicitar_avaliacao(pergunta: str, resposta: str, topico: str) -> Dict:
    """Solicita avaliação da resposta ao Agente Avaliador."""
    avaliador = _estado["agente_avaliador"]

    if avaliador is None:
        return {
            "nivel": "sabe_parcial",
            "justificativa": "Avaliador não disponível",
            "proximo_passo": "avançar"
        }

    # Atualizar tracking
    if topico not in _estado["tracking"]:
        _estado["tracking"][topico] = {"tentativas": 0}

    _estado["tracking"][topico]["tentativas"] += 1

    return {
        "nivel": "sabe_parcial",
        "justificativa": "Avaliação realizada com base no material",
        "proximo_passo": "avançar"
    }

@tool
def exibir_feedback(feedback: str) -> str:
    """Exibe feedback BREVE ao aluno na tela."""
    print(f"\n💬 TUTOR: {feedback}\n")

    _estado["historico"].append({
        "tipo": "feedback",
        "conteudo": feedback
    })

    return "Feedback exibido ao aluno"

@tool
def obter_contexto_recente() -> Dict:
    """Obtém contexto recente para formular pergunta contextual."""
    ultimos_items = _estado["historico"][-5:] if len(_estado["historico"]) > 5 else _estado["historico"]

    contexto = []
    for item in ultimos_items:
        if item["tipo"] == "pergunta":
            contexto.append(f"PERGUNTA: {item['conteudo']}")
        elif item["tipo"] == "resposta":
            contexto.append(f"RESPOSTA: {item['conteudo']}")
        elif item["tipo"] == "feedback":
            contexto.append(f"FEEDBACK: {item['conteudo']}")

    return {
        "ultimas_interacoes": "\n".join(contexto) if contexto else "Nenhuma interação ainda",
        "perguntas_restantes": _estado["max_perguntas"] - _estado["num_perguntas"]
    }

@tool
def obter_historico_completo() -> Dict:
    """Obtém TODO o histórico para gerar relatório final."""
    historico_formatado = []

    for item in _estado["historico"]:
        if item["tipo"] == "pergunta":
            historico_formatado.append(f"PERGUNTA {item.get('numero', '?')}: {item['conteudo']}")
        elif item["tipo"] == "resposta":
            historico_formatado.append(f"RESPOSTA DO ALUNO: {item['conteudo']}")
        elif item["tipo"] == "feedback":
            historico_formatado.append(f"FEEDBACK DADO: {item['conteudo']}")

    return {
        "historico_formatado": "\n\n".join(historico_formatado),
        "estatisticas": {
            "total_perguntas": _estado["num_perguntas"],
            "max_perguntas": _estado["max_perguntas"]
        }
    }

@tool
def salvar_relatorio(conteudo: str) -> str:
    """Salva relatório em arquivo markdown."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_arquivo = f"relatorio_aprendizagem_{timestamp}.md"

    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write(conteudo)

    print("\n" + "="*70)
    print("RELATÓRIO FINAL DE APRENDIZAGEM")
    print("="*70 + "\n")
    print(conteudo)
    print("\n" + "="*70)
    print(f"Relatório salvo em: {nome_arquivo}")
    print("="*70 + "\n")

    return f"Relatório exibido e salvo: {nome_arquivo}"

# Tool para registrar decisão (APRESENTAR)
@tool
def registrar_decisao(decisao: str, topico: str, motivo: str) -> str:
    """
    Registra decisão do Coordenador sobre próximo passo.

    Args:
        decisao: 'avançar', 'aprofundar', ou 'encerrar'
        topico: Tópico atual
        motivo: Justificativa da decisão

    Returns:
        Confirmação do registro
    """
    if topico in _estado["tracking"]:
        _estado["tracking"][topico]["decisao"] = decisao
        _estado["tracking"][topico]["motivo"] = motivo

        # Se avançar, marcar status final
        if decisao == "avançar":
            ultima_aval = _estado["tracking"][topico].get("ultima_avaliacao", {})
            nivel = ultima_aval.get("nivel", "sabe_parcial")

            if nivel == "nao_sabe":
                _estado["tracking"][topico]["status"] = "lacuna"
            elif nivel == "sabe_parcial":
                _estado["tracking"][topico]["status"] = "parcial"
            else:
                _estado["tracking"][topico]["status"] = "dominado"

    return f"✓ Decisão registrada: {decisao} ({motivo})"

# Função para configurar estado (APRESENTAR)
def configurar_estado(agente_avaliador, max_perguntas=10):
    """
    Configura estado compartilhado (chamado pelo orchestrator)

    Args:
        agente_avaliador: Agente avaliador para usar nas tools
        max_perguntas: Número máximo de perguntas
    """
    _estado["agente_avaliador"] = agente_avaliador
    _estado["max_perguntas"] = max_perguntas

    print(f"✅ Estado configurado!")
    print(f"📊 Agente avaliador: {agente_avaliador.name}")
    print(f"📊 Max perguntas: {max_perguntas}")


# 🚀 Execução do Sistema Agêntico

> *Vamos ver nossos agentes trabalharem juntos!*

Esta é a etapa de "montagem" final. Estamos atribuindo a lista de funções-ferramenta à propriedade .tools do nosso agente Coordenador.

In [ ]:
coordenador.tools = [
    obter_contexto_recente,
    fazer_pergunta,
    solicitar_avaliacao,
    exibir_feedback,
    obter_historico_completo,
    salvar_relatorio,
    registrar_decisao
]

> Este é o gatilho que inicia todo o processo. Damos uma instrução inicial ao Coordenador ("Inicie a avaliação..."). A partir daí, o agente se torna autônomo

In [ ]:

resultado = coordenador.run(f"Inicie a avaliação agêntica do aluno sobre o material fornecido.\n"
          f"Você tem total autonomia para conduzir o processo.\n"
          f"Limite de perguntas: {_estado["max_perguntas"]}\n\n"
          f"INSTRUÇÃO ESPECÍFICA: Comece AGORA fazendo a primeira pergunta!\n"
          f"Obtenha o contexto recente para ver o estado atual e então faça sua primeira pergunta.")




# 🎉 Resumo do Checkpoint 3

## O que aprendemos:
- **Como fazer agentes trabalharem juntos**
- **Como integrar tools com agentes**
- **Executar um sistema completo**
- **Resultados em tempo real**





